In [4]:
import requests
import requests_cache
import numpy as np
import pandas as pd
import lxml.html as lxl
import nltk
import re
from nltk.corpus import stopwords
from datetime import datetime

In [5]:
requests_cache.install_cache('demo_cache')

In [15]:
def urls_scraping(base_url, search_term):
    try:
        response = requests.get(base_url, params={'searchterms': serach_term})
        response.raise_for_status()
    except:
        return
    
    lxl.fromstring(response.content)
    
def page_scraping(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except:
        return
    
    # extract title
    title = root.xpath('//div[@class="job-title"]/h1/text()')[0].strip()     
    
    # extract city and location of that posting.
    location = [i.strip() for i in root.xpath('//div[@class="location"]/span/text()')[0].split(',')]
    city = location[0]
    state = location[1]
    
    # extract job type and salary(to be done)
    type_salary = [int(i.replace('k', '000')) for i in re.findall(r'\d+k',root.xpath('//div[@class="wage"]/span/text()')[0] ) ] 
    if type_salary == []:
        type_salary = 'NA'
    # extract preferred skills(string type)
    skill_ls = root.xpath('//div[@class="skills"]/ul[@class="skill-list"]                         \
           /li[@class="skill-item"]/a/span[@class="skill-name"]/text()')
    preferred_skill = ', '.join([skill.strip() for skill in skill_ls])
    
    # extract 'what you need for this position'
    need_for_position = root.xpath('//div[@class="section-data section-data-title" and @data-section="7"]/text()')
    
    #extract job id
    job_id = root.xpath('//div[@class="job-id"]/text()')[0].strip()
    job_id = re.sub('.*: ','', job_id)  
    
    #extract post date
    post_date = root.xpath('//div[@class="mobile-hide posted posted-text"]/span/text()')[0].strip()
    
    if post_date == 'Posted Today':
        post_date = datetime.today().strftime('%m/%d/%Y')
    else:
        post_date = re.sub(r'^Posted ([0-9]{2}/[0-9]{2}/[0-9]{4})', r'\1', post_date)
    
    return {'city':city, 'state': state, 'salary': type_salary, 'preferred_skill':preferred_skill, 'need_for_position': need_for_position,
           'job_id': job_id, 'post_date': post_date}

In [12]:
response = requests.get('https://www.cybercoders.com/systems-administrator-job-327437')
response.raise_for_status()
root = lxl.fromstring(response.content)

In [13]:
root.xpath('//div[@class="section-data section-data-title" and @data-section="7"]/text()')

['Must HAVE Than 5 Years of experience and knowledge of:',
 '- Systems Administration',
 '- Linux',
 '- Windows Systems Administration',
 '- Cloud Storage',
 '- Active Directory']

In [16]:
a = root.xpath('//div[@class="mobile-hide posted posted-text"]/span/text()')[0].strip()

In [17]:
page_scraping('https://www.cybercoders.com/systems-administrator-job-327437')

{'city': 'Nashville',
 'job_id': 'DS6-1347854',
 'need_for_position': ['Must HAVE Than 5 Years of experience and knowledge of:',
  '- Systems Administration',
  '- Linux',
  '- Windows Systems Administration',
  '- Cloud Storage',
  '- Active Directory'],
 'post_date': '02/18/2017',
 'preferred_skill': 'Systems Administration, Linux, Windows Systems Administration, Cloud Storage, Active Directory, vmware, MySQL',
 'salary': [60000, 90000],
 'state': 'TN'}